In [1]:
 #Feature Engineering


  # Clean and pre-process the text data.
  # Define features for comparing the similarity of an answer text and a source text, and extract similarity features.
  # Select "good" features, by analyzing the correlations between different features.
  # Create train/test .csv/.txt files that hold the relevant features and class labels for train/test data points.


In [2]:
# import libraries
import pandas as pd
import numpy as np
import os
import operator 
import re
import sys

from adaptnlp import (
    EasyTokenTagger,
    EasySequenceClassifier,
    EasyQuestionAnswering,
    EasySummarizer,
    EasyTranslator,
    EasyDocumentEmbeddings,
)

In [3]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
from nltk.stem import PorterStemmer
from collections import Counter
import string
from scipy.sparse import hstack


from featurefunctions import create_text_column, features,normalized_features,count_features,tfidf_features,tfidf_ngram_features,tfidf_ngram_chars,ling_feature_vec,all_count_features,all_features

Using TensorFlow backend.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swcam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\swcam\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\swcam\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## Import Data


In [4]:
##TODO

csv_file = '/Users/swcam/Documents/Github/Sarcasm/SarcasmTwitterTraining.csv'

#'/Users/swcam/Documents/Github/Sarcasm/SarcasmTwitterTraining.csv'
#csv_file = '/content/drive/My Drive/Colab Notebooks/Novetta_Disinformation/News_dataset/total_text_data.csv' 
df = pd.read_csv(csv_file)

# print out the first few rows of data info
df.head(2)

,label,response,context/0,context/1,context/2,context/3,context/4,context/5,context/6,context/7,...,context/10,context/11,context/12,context/13,context/14,context/15,context/16,context/17,context/18,context/19
0,SARCASM,@USER @USER @USER I don't get this .. obviousl...,A minor child deserves privacy and should be k...,@USER If your child isn't named Barron ... #Be...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SARCASM,@USER @USER trying to protest about . Talking ...,@USER @USER Why is he a loser ? He's just a Pr...,@USER @USER having to make up excuses of why y...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
new_df = df.drop(['context/2','context/3','context/4','context/5','context/6','context/7','context/8','context/9','context/10','context/11','context/12','context/13','context/14','context/15','context/16','context/17'], axis=1)


In [6]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion")

model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion")

def get_emotion(text):
  input_ids = tokenizer.encode(text + '</s>', return_tensors='pt')

  output = model.generate(input_ids=input_ids,
               max_length=2)

  dec = [tokenizer.decode(ids) for ids in output]
  label = dec[0]
  return input_ids

get_emotion("i feel as if i havent blogged in ages are at least truly blogged i am doing an update cute") # Output: 'joy'

get_emotion("i have a feeling i kinda lost my best friend") # Output: 'sadness'

tensor([[   3,   23,   43,    3,    9, 1829,    3,   23,  773,    9, 1513,   82,
          200, 1565,    1]])

In [7]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion")

model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion")

def get_emotion_label(text):
  input_ids = tokenizer.encode(text + '</s>', return_tensors='pt')

  output = model.generate(input_ids=input_ids,
               max_length=2)

  dec = [tokenizer.decode(ids) for ids in output]
  label = dec[0]
  return label

get_emotion_label("i feel as if i havent blogged in ages are at least truly blogged i am doing an update cute") # Output: 'joy'

get_emotion_label("i have a feeling i kinda lost my best friend") # Output: 'sadness'

'sadness'

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1,feature_vec_2):
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

In [8]:
def get_vectors(text1, text2):
    # Turns text into emotional tensors
    tensor1 = get_emotion(text1)
    tensor2 = get_emotion(text2)
    
    # Turns emotional tensors into numpy array
    Vector_1 = numpy.array(tensor1)
    Vector_2 = numpy.array(tensor2)
    
    #pads lower array with zeros to match higher array
    if Vector_1.size >= Vector_2.size:
        vA = Vector_1
        vB = Vector_2
        result = np.zeros(vA.shape)
        result[:vB.shape[0],:vB.shape[1]] = vB
        vC = vA
    elif Vector_1.size <= Vector_2.size:
        vB = Vector_2
        vA = Vector_1
        result = np.zeros(vB.shape)
        result[:vA.shape[0],:vA.shape[1]]= vA
        vC = vB
    return vC, result


    
    

In [9]:
text2 = '@USER @USER trying to protest about . Talking about him and his labels and they label themselves WTF does that make em ?'
text1 = 'NaN'

In [10]:
emotion = get_emotion_label(text2)

NameError: name 'get_emotion_label' is not defined

In [36]:
a,b = get_vectors(text2,text1)

In [37]:
cosine_similarities = get_cosine_similarity(a,b)

In [38]:
print(cosine_similarities)

0.2694203211894635


In [ ]:
# creating lists to keep pos, neu, neg, and compound scores --- later to be used to create a dataframe
response_sent = []



# extracting vader scores for each entry in the data
# (we're not using context yet.)
# note that the compound score is rescaled to the [0,1] range
# some classiifers don't take negative values (e.g., MultinomialNB)

for row in new_df["response"]:
    sentiment = get_emotion_label(row)
  
    response_sent.append(sentiment)


##Create feature vector 
new_df["Response_emotion"] = response_sent


In [110]:
# creating lists to keep pos, neu, neg, and compound scores --- later to be used to create a dataframe
context_sent = []



# extracting vader scores for each entry in the data
# (we're not using context yet.)
# note that the compound score is rescaled to the [0,1] range
# some classiifers don't take negative values (e.g., MultinomialNB)

for row in new_df["context/0"]:
    if row == 0:
        
        sentiment = 'NaN'
  
        context_sent.append(sentiment)
    else:
        
        sentiment = get_emotion_label(row)
        context_sent.append(sentiment)
        


##Create feature vector 
new_df["context0_sentiment"] = context_sent


In [ ]:
### Should Theoretically work as long as get_vectors function is working properly 


Emotional_sim = []
for row_a in new_df['response']:
    if row_a == 0:
        row_a = 'NaN'
    else:
        row_a = row_a
            
    for row_b in new_df['context/0']:
        if row_b == 0:
            row_b = 'NaN'
        else:
            row_b = row_b
        
        
        text1 = row_a
        text2 = row_b
        
    
        a,b = get_vectors(text1,text2)
        cosine_similarities = get_cosine_similarity(a,b)
        Emotional_sim.append(cosine_similarities)
    
new_df["Emotional_sim_response/context/0"] = Emotional_sim


In [111]:
new_df.head()

,label,response,context/0,context/1,context/18,context/19,Response_emotion,context0_sentiment
0,SARCASM,@USER @USER @USER I don't get this .. obviousl...,A minor child deserves privacy and should be k...,@USER If your child isn't named Barron ... #Be...,NaN,NaN,joy,sadness
1,SARCASM,@USER @USER trying to protest about . Talking ...,@USER @USER Why is he a loser ? He's just a Pr...,@USER @USER having to make up excuses of why y...,NaN,NaN,anger,sadness
2,SARCASM,@USER @USER @USER He makes an insane about of ...,Donald J . Trump is guilty as charged . The ev...,@USER I ’ ll remember to not support you at th...,NaN,NaN,anger,anger
3,SARCASM,@USER @USER Meanwhile Trump won't even release...,Jamie Raskin tanked Doug Collins . Collins loo...,@USER But not half as stupid as Schiff looks ....,NaN,NaN,anger,sadness
4,SARCASM,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,Man ... y ’ all gone “ both sides ” the apocal...,@USER They already did . Obama said many times...,NaN,NaN,anger,sadness


# Emotional Similiarity 

In [52]:
### Should Theoretically work as long as get_vectors function is working properly 


Emotional_sim = []
for row_a in new_df['response']:
    if row_a == 0:
        row_a = 'NaN'
    else:
        row_a = row_a
            
    for row_b in new_df['context/0']:
        if row_b == 0:
            row_b = 'NaN'
        else:
            row_b = row_b
        
        
        text1 = row_a
        text2 = row_b
        
    
        if text1 or text2 == 'NaN':
            cosine_similarities = 'NaN'
            Emotional_sim.append(cosine_similarities)
        
        else:
            a,b = get_vectors(text1,text2)
            cosine_similarities = get_cosine_similarity(a,b)
            Emotional_sim.append(cosine_similarities)
    
new_df["Emotional_sim_response/context/0"] = Emotional_sim


ValueError: Length of values does not match length of index

In [49]:
new_df.head()

,label,response,context/0,context/1,context/18,context/19,Emotional_sim_response/context/0
0,SARCASM,@USER @USER @USER I don't get this .. obviousl...,A minor child deserves privacy and should be k...,@USER If your child isn't named Barron ... #Be...,NaN,NaN,NaN
1,SARCASM,@USER @USER trying to protest about . Talking ...,@USER @USER Why is he a loser ? He's just a Pr...,@USER @USER having to make up excuses of why y...,NaN,NaN,NaN
2,SARCASM,@USER @USER @USER He makes an insane about of ...,Donald J . Trump is guilty as charged . The ev...,@USER I ’ ll remember to not support you at th...,NaN,NaN,NaN
3,SARCASM,@USER @USER Meanwhile Trump won't even release...,Jamie Raskin tanked Doug Collins . Collins loo...,@USER But not half as stupid as Schiff looks ....,NaN,NaN,NaN
4,SARCASM,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,Man ... y ’ all gone “ both sides ” the apocal...,@USER They already did . Obama said many times...,NaN,NaN,NaN


In [ ]:
new_df.to_csv("Emotional_similiarity.csv")

In [38]:
df.groupby(['label', 'Response_sentiment']).size().reset_index(name='counts')

,label,Response_sentiment,counts
0,NOT_SARCASM,anger,221
1,NOT_SARCASM,fear,42
2,NOT_SARCASM,joy,481
3,NOT_SARCASM,love,25
4,NOT_SARCASM,sadness,125
5,NOT_SARCASM,surprise,6
6,SARCASM,anger,340
7,SARCASM,fear,49
8,SARCASM,joy,380
9,SARCASM,love,12
